<a href="https://colab.research.google.com/github/silvia-jesus/CAR_Labrea/blob/main/CAR_RL_Labrea_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from pathlib import Path

In [ ]:
from osgeo import gdal

In [ ]:
! pip install geopandas
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-4-115f0ca902c9>:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  impo

In [ ]:
!sudo apt install libspatialindex-dev
!pip install rtree
!pip install pygeos

import rtree
import pygeos
gpd.options.use_pygeos = True
USE_PYGEOS=1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.9.3-1build1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3249: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
!pip install rasterio
import rasterio as rio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rasterstats
from rasterstats import zonal_stats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rasterio import mask

In [ ]:
!pip install fiona
import fiona

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
import glob
import shutil
import os
import time

In [ ]:
if not os.path.exists('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC_RECODE'):
  os.makedirs('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC_RECODE')

Path('./').mkdir(parents=True, exist_ok=True)

In [ ]:
if not os.path.exists('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC_RECODE/Arrays_LULC'):
  os.makedirs('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC_RECODE/Arrays_LULC')

Path('./').mkdir(parents=True, exist_ok=True)

In [ ]:
recode6=glob.glob('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/*')
recode6

['/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2020_recode.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2008_recode.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2012_recode.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2016_recode.tif']

In [ ]:
files_rl = glob.glob('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_SPLIT/*.shp')
len(files_rl)

2860

In [ ]:
start = time.time()
for rl in files_rl:
    for lulc in recode6:
        with fiona.open(rl, 'r') as shapefile:
            shapes = [feature['geometry'] for feature in shapefile]
        with rio.open(lulc) as src:
            out_image, transformed = rio.mask.mask(src, shapes, crop=True, filled=True)

            gdf = gpd.read_file(rl)

            id_rl=gdf.iloc[0]['IDF']
            year = lulc.replace('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/','').replace('_recode.tif','')

            data=out_image.flatten()
            data=data[data >= 0] #to remove nan
            unique, counts = np.unique(data, return_counts=True)
            arr = np.array([unique, counts])
            df = pd.DataFrame(data=arr)
            new_header = df.iloc[0] #grab the first row for the header
            df = df[1:] #take the data less the header row
            df.columns = new_header #set the header row as the df header

            df['IDF']=id_rl
            df['ano']=year
            df.to_csv(f'/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC_RECODE/Arrays_LULC/recode_{id_rl}_{year}.csv')

end = time.time()
duration = end-start
print(f'O processamento demorou {duration} segundos.')

O processamento demorou 6310.745069503784 segundos.


In [ ]:
lulc_csv = glob.glob('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC_RECODE/Arrays_LULC/recode_*.csv')

In [ ]:
lulc_df = pd.concat(map(pd.read_csv, lulc_csv), ignore_index=True)
lulc_df = lulc_df.iloc[:, 2:] #remove old index and 0 values

cols = lulc_df.columns
cols

Index(['1', '3', 'IDF', 'ano', '2', '5'], dtype='object')

In [ ]:
lulc_df = lulc_df.fillna(0)
lulc_df.head(6)

,1,3,IDF,ano,2,5
0,1238.0,463.0,113026,2008,0.0,0.0
1,1260.0,441.0,113026,2016,0.0,0.0
2,1225.0,476.0,113026,2012,0.0,0.0
3,1355.0,346.0,113026,2020,0.0,0.0
4,315.0,33.0,113802,2008,0.0,0.0
5,121.0,227.0,113802,2016,0.0,0.0


In [ ]:
lulc_df = pd.concat(map(pd.read_csv, lulc_csv), ignore_index=True)
lulc_df = lulc_df.iloc[:, 2:] #remove old index and 0 values
cols = lulc_df.columns
cols

Index(['1', '3', 'IDF', 'ano', '2', '5'], dtype='object')

In [ ]:
if not os.path.exists('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/Tabelas_LULC'):
  os.makedirs('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/Tabelas_LULC')


Path('./').mkdir(parents=True, exist_ok=True)

In [ ]:
lulc_df.to_csv('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/Tabelas_LULC/RL_RECODE_Labrea.csv')